In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
sourcedir = os.path.join('..','sourcedata')
outputdir = os.path.join('..','derivatives')

In [3]:
fpath = os.path.join(sourcedir,'memrl_data.xlsx')
data = pd.read_excel(fpath)

In [4]:
data.head()

,SubjNum,AgeGroup,ExperimenterName,RunNum,Date,Time,TrialNum,DelayCond,TrialNumbyCue,CueOnLeft,...,ButtonResp,Outcome,OutcomePic,CueStart,RT,OutcomeStart,OutcomeDuration,IntertrialDuration,ChoseGoodCue?,TrialBlock
0,101,1,nl,0,10_28,0.719317,1,1,1,1,...,1.0,1.0,indoor_(45).jpg,4.033720,0.929655,6.000884,1.506218,2.779541,0.0,1
1,101,1,nl,0,10_28,0.719317,2,1,2,0,...,1.0,0.0,outdoor_(81).jpg,10.286723,0.931679,12.252411,1.506221,3.537933,1.0,1
2,101,1,nl,0,10_28,0.719317,3,2,1,1,...,1.0,0.0,outdoor_(67).jpg,17.298038,1.295149,25.645838,1.507751,3.026509,0.0,1
3,101,1,nl,0,10_28,0.719317,4,1,3,1,...,6.0,1.0,indoor_(122).jpg,30.180110,1.188015,32.398769,1.502938,2.776548,1.0,1
4,101,1,nl,0,10_28,0.719317,5,1,4,1,...,6.0,0.0,outdoor_(270).jpg,36.678267,1.085774,38.815834,1.505155,2.776437,1.0,1
